# Astropy Coordinates Introduction


In the end, I'll show a new method of speeding up astropy coordinate transforms I developed together with Benjamin Winkel (MPI Radioastronomie Bonn).

This is merged but not yet released, it will be part of astropy 4.2.

In [ ]:
import astropy

astropy.__version__  # I will be using the current astropy master, not yet released

In [ ]:
# general imports
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt


%config InlineBackend.figure_formats = ['svg']
%matplotlib inline


In [ ]:
plt.rcParams['figure.figsize'] = (9, 6)

## Planning an observation

Let's say we want to observe several well known sources with the LST-1 tonight, when do we observe which source?


First, some definitions up front, the location:

## Observatory location

In [ ]:
from astropy.coordinates import EarthLocation

In [ ]:
EarthLocation.get_site_names()[55:65]

In [ ]:
# Somewhere near the William Herrschel Telescope
orm = EarthLocation.of_site('Roque de los Muchachos')

# more precise (From Google Earth)
lst1 = EarthLocation(lat=28.761466 * u.deg, lon=-17.891541 * u.deg, height=2187 * u.m)

orm.geodetic, lst1.geodetic

## Time

In [ ]:
from astropy.time import Time

start = Time('2020-10-09T17:00Z')
start

In [ ]:
start.ut1, start.utc, start.tai, start.tt

In [ ]:
# let's get some points through the night in 5 minute intervals
obstime = start + np.arange(0, 16 * 60, 5) * u.min

## Sources

* Sources live in the skyfixed, equatorial coordinate frame: `ICRS` / `ICRF`, the default frame of `SkyCoordinate`.

* Astropy also supports older reference frames, e.g. `FK5` which then needs the equinox, e.g. `J2000.0`

`SkyCoordinate` can query a database of objects using `from_name`, which is quite handy:

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
source_names = ['Crab', 'Mrk 501', 'Mrk 421', 'NGC 1275', 'IC310']

In [ ]:
sources_icrs = {
    name: SkyCoord.from_name(name)
    for name in source_names
}

sources_icrs['Crab']

### Transform to Horizontal Coordinates (AltAz)

Astropy uses a graph of coordinate frame transformation, to enable transformation from and to any frame,
as long as all needed information (frame attributes) are available.

In [ ]:
from astropy.coordinates import AltAz

# first for one time, just to take a look
altaz = AltAz(obstime=start, location=lst1)

crab_altaz = sources_icrs['Crab'].transform_to(altaz)

# astropy primarily uses alt, but also provides zen
crab_altaz.alt, crab_altaz.az, crab_altaz.zen

In [ ]:
# now for all times and sources
altaz = AltAz(obstime=obstime, location=lst1)

sources_altaz = {n: s.transform_to(altaz) for n, s in sources_icrs.items()}

In [ ]:
# Let's also get the sun and moon to make sure it's not too bright

from astropy.coordinates import get_sun, get_moon

sun = get_sun(obstime).transform_to(altaz)
moon = get_moon(obstime).transform_to(altaz)

In [ ]:
fig, ax = plt.subplots()

x = obstime.to_datetime()

for name, source_altaz in sources_altaz.items():
    ax.plot(
        x,
        source_altaz.zen.to_value(u.deg),
        label=name,
    )
    
ax.plot(x, sun.zen.deg, color='xkcd:yellow', label='Sun')
ax.plot(x, moon.zen.deg, color='gray', label='Moon')


# Fill twilight zones

ax.fill_between(x, 0, 120, where=(sun.alt.deg > -18), color='k', alpha=0.5, lw=0)
ax.fill_between(x, 0, 120, where=sun.alt.deg > -12, color='k', alpha=0.3, lw=0)

ax.set_yticks([0, 30, 60, 90, 102, 108])
ax.set_yticklabels([0, 30, 60, 90, 'Nautical dusk/dawn', 'Astronomical dusk/dawn'])
ax.yaxis.grid()

ax.legend(ncol=len(source_names) + 2, bbox_to_anchor=[0.5, 1.01], loc='lower center')
ax.set_ylim(120, 0)
ax.set_xmargin(0)
fig.autofmt_xdate()

## Speeding Coordinate Transformations using Interpolation


Slow transformation times for many (thousands to millions) of coordinates close in time were a major pain point for use with IACTs.

LST: 10.000 Events per second, almost all need the transformation from AltAz to ICRS after the event reconstruction.

* First PR: https://github.com/astropy/astropy/pull/6068 (2017, Benjamin Winkel)

  Interpolation and non-interpolation case was inter-twined and not easy to reason about.  
  ⇒ Progress stalled for 3 years.

* Second PR: https://github.com/astropy/astropy/pull/10647 (2020, by me).  
  
  Refactored to cleanly separate the default from the interpolating approach. Added examples, docs and tests.
  Merged last month.

In [ ]:
# random lst events, 100 000 events in the field of view

alt = np.random.normal(70, 0.1, 100_000) * u.deg
az = np.random.normal(0, 0.1, len(alt)) * u.deg
obstime = Time('2020-10-01T02:00') + np.random.uniform(0, 10, len(alt)) * u.min

altaz = SkyCoord(alt=alt, az=az, frame=AltAz(obstime=obstime, location=lst1))

In [ ]:
%%time

icrs = altaz.transform_to('icrs')

## Excursion: profiling

What exactly is so slow here?

In [ ]:
%%prun -r -q -D coordinates.perf


icrs = altaz.transform_to('icrs')

In [ ]:
profile = _

In [ ]:
profile.strip_dirs().sort_stats('tottime').print_stats(20)

## The interpolation solution

We saw that earth position and velocity take the most time (Transformation from Solarsystem barycenter to 

In [ ]:
from astropy.coordinates.erfa_astrom import erfa_astrom, ErfaAstromInterpolator

In [ ]:
%%time

with erfa_astrom.set(ErfaAstromInterpolator(time_resolution=5 * u.min)):
    
    icrs_interpolated = altaz.transform_to('icrs')

In [ ]:
((15.9 * u.s) / (175 * u.ms)).to(u.one)

In [ ]:
icrs_interpolated.separation(icrs).max().to_value(u.microarcsecond)